In [8]:
# This notebook extract motions previously manually extracted from public whip and stores as a yaml

from pathlib import Path
import os
import re
import pandas as pd
from datetime import date
from ruamel.yaml import YAML

while not (Path.cwd() / "pyproject.toml").exists():
    os.chdir("..")

from twfy_votes.internal.db import duck_core
from twfy_votes.apps.legacy.data_sources import duck as tables_duck


duck = await duck_core.get_core()

await duck.compile(tables_duck).run()

In [9]:
query = """
SELECt
    division_date,
    division_number,
    house as chamber,
    last(text_body) as text_body,
FROM
    pw_dyn_wiki_motion
GROUP BY
    all

"""


def extract_motion_text(item: str) -> str:
    # get text between --- MOTION EFFECT --- and --- COMMENTS AND NOTES ---
    try:
        return (
            item.split("--- MOTION EFFECT ---")[1]
            .split("--- COMMENTS AND NOTES ---")[0]
            .strip()
            .replace("\r\n\r\n", "\n")
        )
    except IndexError:
        return item


df = await duck.compile(query).df()

df["manual_motion"] = df["text_body"].apply(extract_motion_text)
df = df.drop(columns=["text_body"])


items = df.to_dict("records")

dest = Path("data", "divisions", "manual_motions.yaml")

yaml = YAML()
yaml.default_flow_style = False
yaml.dump(items, dest)